In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('../styles/custom.css', 'r').read()
    return HTML(styles)
css_styling()

# Lifting bodies

This section will modify the vortex panel method to enable the prediction of lift on wing sections.

## Foil section geometry

To find the lift on a foil, we first need to define the geometry. Let's use a symmetric Jukowski foil for now. This foil shape is obtained by applying the following mapping

$$ x' = \frac x2 (1+1/r^2), \quad y' = \frac y2 (1-1/r^2),\quad r^2= x^2+y^2 $$

to the points on a circle which intersects $x,y=1,0$. Shifting the center of the circle an amount $\Delta x$ sets the thickness of the foil. I've followed this procedure in the function `make_jukowski`. Let's import it (along with the rest of `VortexPanel`):

In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline
import VortexPanel as vp
help(vp.make_jukowski)

The arguments are a number of points $N$, the x-shift `dx`$=\Delta x$, and similar shifts to the angle `dtheta`$=\Delta \theta$ and radius `dr`$=\Delta r$. Let's test it out.

In [ ]:
foil = vp.make_jukowski(N=32)

pyplot.figure(figsize=(10,4))
pyplot.axis([-1.2,1.2,-.5,.5])
for p in foil: p.plot()

##### Quiz

How can you get a cambered (ie, curved) foil shape using the Jukowski transform?

1. Shift the center right or left.
1. Rotate the circle around (1,0).
1. Change the circle's radius.
---

We're going to use the symmetric foil with a sharp edge - so put things back when you're done.

## Flows with circulation

Now lets solve for the flow around the foil:

In [ ]:
vp.solve_gamma(foil)     # solve for gamma
vp.plot_flow(foil)       # plot the flow

Whoops. We need an angle of attack. *Notice that I need to use the same `alpha` for both solving and plotting.* 

In [ ]:
alpha = numpy.pi/16            # set angle of attack
vp.solve_gamma(foil,alpha)     # solve for gamma
vp.plot_flow(foil,alpha)       # plot the flow

Sigh. There are so many things wrong with this picture...

 - The flow is faster on the underside of the foil. 
 - The rear stagnation point is on the upper side of the foil.
 - There is a singularity on the lower side of the trailing edge.

These are all physically incorrect! But how can setting the (physical) no-slip condition on each panel lead to non-physical results?

It is a symptom that the vortex panel no-slip matrix is *singular*.

##### Mathematics fundamental: Singular matrix
##### The solutions to a singular matrix are not unique

This means there are many (non-physical) solutions for this flow, one of which is given above.

## The Kutta condition

However, there **is** a unique (physical) solution for the foil, and this is because the foil has a sharp trailing edge.

##### Quiz

What does the flow need to look like on the trailing edge?

![short](resources\trailingEdge.png)

1. The flow needs to wrap from the bottom to the top.
1. The flow needs to wrap from the top to the bottom.
1. The flow needs to separate from the edge. 

---
The fundamental problem with the solution for the foil above is that the flow is wrapping around the sharp trailing edge. This means that particles traveling on the body streamline *instantly* change direction - which is impossible for any object with mass.

Avoiding this impossibility is called the Kutta condition.

##### Hydrodynamics fundamental: Kutta Condition
##### Potential flow must separate from a sharp trailing edge

This uniquely determines the correct solution $\gamma_i$ and it links the solution to the angle of attack $\alpha$ to the lift. As $\alpha$ is increased, the amount of circulation increases to enforce the Kutta condition.

To enforce the Kutta condition in the vortex panel method, we need to convert it into a statement about the strength $\gamma(s)$. 

##### Quiz

What condition on $\gamma(s)$ will enforce the Kutta conditions?

1. $\gamma=U_\infty$ near the trailing edge
1. $\gamma=0$ near the trailing edge
1. $\gamma$ is antisymmetric near the trailing edge, ie $\gamma(-s)=-\gamma(s)$

(Hint: Look at the sketch above. What does it tell you about $\gamma$ near the trailing edge?)

---

## Adjusted linear system

Now we know the additional condition which makes the solution unique. You may have noticed that `solve_gamma` takes a kutta argument...

In [ ]:
help(vp.solve_gamma)

Like in the example, the two panels on either side of the trailing edge are `(0,N-1)`, so we can use
```python
vp.solve_gamma(foil,alpha,kutta=[(0,-1)])
```
or we can use the `solve_gamma_kutta` function, which does exactly the same thing:

In [ ]:
help(vp.solve_gamma_kutta)

## Quantitative testing

To test if this function really does apply the kutta condition, let's plot $\gamma(s)$, where $s$ is the distance around the foil. After `solve`ing we need two things:
 - `gamma` as an array
 - `s` as an array

We will use the `VortexPanel.get_array(panels,'key')` function for the first, which gets arrays of one or more attributes from `panels`. 

For the second, we can `get_array` of the panel half-widths `S`, and take their cumulative sum to get the path distance around the foil:
```python
S = vp.get_array(panels,'S')
return numpy.cumsum(2*S)-S
```
which is implemented as the `VortexPanel.distance` function. 

As always, you can use `help` to see the details of these two functions.

---

Alright, let's test the impact of the kutta condition:

In [ ]:
alpha = numpy.pi/16           # angle of attack
foil = vp.make_jukowski(N=32) # make geometry
s = vp.distance(foil)         # distance array

# solve without kutta and plot 
vp.solve_gamma(foil,alpha)              # solve
gamma = vp.get_array(foil,'gamma')      # get
pyplot.plot(s,gamma,label="No kutta")   # plot

# solve with kutta and plot
vp.solve_gamma_kutta(foil,alpha)        # solve
gamma = vp.get_array(foil,'gamma')      # get
pyplot.plot(s,gamma,label="With kutta") # plot

# finish gamma(s) plot
pyplot.legend()
pyplot.xlabel(r'$s$', fontsize=20)
pyplot.ylabel(r'$\gamma$', fontsize=20)
pyplot.axhline(0,c='k',ls='--')
pyplot.show()

# plot flow
vp.plot_flow(foil,alpha)

The quantitative comparison of $\gamma(s)$ shows that the Kutta condition has been sucsessfully enforced, and that this has completely changed the solution. 

The flow plot shows that this has corrected all the non-physical features:
- High speed flow on the top, leading to positive lift.
- Clean separation from the trailing edge.
- The circulation is unique for a given angle of attack.

## Lift force

Speaking of lift... we could determine the lift by evaluating a pressure integral

$$ L = -\oint p n_y da $$

However, the Kutta-Joukowski lift force on a body in potential flow is even easier:

$$ L = -\rho U_\infty \Gamma $$

where $\Gamma$ is the total circulation. For a continuous vortex sheet this is given by $\Gamma=\oint\gamma(s)ds$.

We non-dimensionalize the lift as

$$ C_L =\frac L{\tfrac 12 \rho U_\infty^2 c} $$

where $c$ is the coord length, the distance from the leading to trailing edge.


---

##### Quiz

What is the expression for the lift coefficient in terms `gamma` and the panel half-width `S`?
- `-sum(gamma)/(2*c)`
- `-sum(gamma*2*S)/(0.5*c*U_inf)`
- `-sum(gamma)*2*S/0.5*c*U_inf`
---


## Numerical validation

Before we do that, how will we know if our lift predictions are any good?

##### Numerical fundamental: Validation
##### A method is validated by comparing the result to a known solution

The reason I implemented a Jukowski foil is not because it is a great foil shape (it isn't). It is because there is an exact solution!

$$C_L = 2\pi \left(1+\frac 4{3\sqrt 3} \frac tc \right)\sin\alpha$$

where $t/c$ is the thickness to coord ratio. You'll use this to test the predictions:

##### Your turn #3

 - ** Complete ** the function `C_L` to compute $C_L$ given a panel array. 
 - ** Complete ** the function `solver_C_L` to compute $C_L$ for a Jukowski foil given an angle of attack. 
 - ** Plot and Compare ** the numerical C_L for $\alpha=0,\ldots,15^o$ against the analytic solution.
 - ** Discuss ** the results. Do they indicate the point of *stall*? Why or why not?

In [ ]:
def C_L(panels):
    # panel attribute arrays
    gamma, xc, S = vp.get_array(panels,'gamma','xc','S')

    # chord length
    c = max(xc)-min(xc)
    
    # return the lift coefficient
    return # your code here

In [ ]:
def solver_C_L(alpha):
    # your code here
    return C_L(foil)

In [ ]:
def analytic_C_L(alpha,t_c=0.2): 
    return 2*numpy.pi*(1+t_c*4/(3*numpy.sqrt(3)))*numpy.sin(alpha)

alpha_deg = numpy.linspace(0,15,6)
alpha = alpha_deg*numpy.pi/180.
pyplot.plot(alpha_deg,analytic_C_L(alpha), label='analytic')

### your code here

pyplot.legend(loc='lower right')
pyplot.xlabel(r'$\alpha$', fontsize=20)
pyplot.ylabel(r'$C_L$', fontsize=20)